In [6]:
import json, requests 


routesurl = "https://syris.iigweb.hevs.ch/en/routes/route_data/2955"

url = requests.get(routesurl)

data = json.loads(url.text)

#user = data[0]
print(data['type'])

features = data['features']
#print(address)

route =[f for f in features if "duration" in f['properties']][0]
print(route)



FeatureCollection
{'type': 'Feature', 'properties': {'title': 'St-Luc - Tignousa par le chemin de Prosper n°4', 'subtitle': 'https://www.outdooractive.com/fr/route/randonnee/saint-luc/st-luc-tignousa-par-le-chemin-de-prosper-n-4/38336270/', 'description': 'Petit chemin qui grimpe entre arolles et mélèzes, pour une virée sportive dans un décor inspirant.', 'duration': '01:45', 'ascent': 529, 'descent': 8, 'tour_length': '4.0', 'typeof': 3, 'ibp_index': 54, 'energy_max': 3, 'technical_max': 2, 'risk_max': 3, 'pk': '2955'}, 'geometry': {'type': 'MultiLineString', 'coordinates': [[[7.597375, 46.220503], [7.597519, 46.220537], [7.597507, 46.220571], [7.597457, 46.220631], [7.597315, 46.220718], [7.597304, 46.220765], [7.597403, 46.220899], [7.597443, 46.221008], [7.597502, 46.221112], [7.597548, 46.221158], [7.597626, 46.221206], [7.597648, 46.221235], [7.597651, 46.221265], [7.597629, 46.221309], [7.59758, 46.22137], [7.597393, 46.221543], [7.597366, 46.2216], [7.597363, 46.221677], [7.597

In [17]:
from rdflib import Graph,Namespace
from rdflib import URIRef,BNode,Literal
from rdflib.namespace import RDF
from rdflib.namespace import RDFS, FOAF

from decimal import Decimal

In [18]:
ANNIVIERS= Namespace("https://hevs.ch/santour/anniviers/")
SANTOUR= Namespace("https://hevs.ch/ontology/santour#")
SCHEMA = Namespace("https://schema.org/")

route_prop=route['properties']

id=route['properties']['pk']

routeIri = URIRef(ANNIVIERS+id)


georoute=route['geometry']['coordinates']
gg=georoute[0]

ls=[[2,3],[4,5]]
lst=[str(i[0])+' '+str(i[1]) for i in ls]



coordlist = [str(coord[0])+' '+ str(coord[1]) for coord in gg]
wkt='LINESTRING('+ ', '.join(coordlist)+')'


g = Graph()


g.bind("schema",SCHEMA)
g.bind("santour",SANTOUR)
g.bind("anniviers",ANNIVIERS)


g.add( (routeIri, RDF.type, SANTOUR.Route ) )
g.add( (routeIri, SCHEMA.name, Literal(route_prop['title']) ) )
g.add( (routeIri, SCHEMA.url, URIRef(route_prop['subtitle']) ) )
g.add( (routeIri, SCHEMA.description, Literal(route_prop['description']) ) )
g.add( (routeIri, SANTOUR.duration, Literal(route_prop['duration']) ) )
g.add( (routeIri, SANTOUR.length, Literal(Decimal(route_prop['tour_length'])) ) )
g.add( (routeIri, SANTOUR.ascent, Literal(int(route_prop['ascent'])) ) )
g.add( (routeIri, SANTOUR.descent, Literal(int(route_prop['descent'])) ) )
      
      
g.add( (routeIri, SCHEMA.geo, Literal(wkt) ) )

#'ibp_index': 54, 'energy_max': 3, 'technical_max': 2, 'risk_max': 3

<Graph identifier=N166c566aa16a4a1e9a246502032d1bb4 (<class 'rdflib.graph.Graph'>)>

In [19]:
from rdflib.serializer import Serializer

context = {"@vocab": "http://purl.org/dc/terms/", 
           "rdfs":"http://www.w3.org/2000/01/rdf-schema#", 
           "owl":"http://www.w3.org/2002/07/owl#",
           "smk":"https://hevs.ch/data/smoking-cessation/",
           "arg" : "https://hevs.ch/ontology/argumentation#", 
           "@language": "en"}
#g.serialize(format='json-ld', context=context, indent=4, destination="../data/arguments.json")
g.serialize(format='turtle', destination="../data/route.ttl")

<Graph identifier=N166c566aa16a4a1e9a246502032d1bb4 (<class 'rdflib.graph.Graph'>)>